In [108]:
import yfinance
import pandas as pd
pd.options.plotting.backend = 'plotly'

import plotly.express as ex

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM

In [18]:
data = yfinance.download('MSFT')

[*********************100%***********************]  1 of 1 completed


In [33]:
startDate = data.index[0].date()
endDate   = data.index[-1].date()
print(f'This Stock Pricres \n From {startDate} \n to   {endDate}')

This Stock Pricres 
 From 1986-03-13 
 to   2023-06-21


In [11]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,9394.000000,9394.000000,9394.000000,9394.000000,9394.000000,9.394000e+03
mean,48.818699,49.346737,48.282761,48.833974,43.439461,5.778076e+07
std,71.498783,72.244066,70.728140,71.527860,71.572115,3.821503e+07
min,0.088542,0.092014,0.088542,0.090278,0.056201,2.304000e+06
25%,5.369141,5.468750,5.304688,5.390625,3.355866,3.350585e+07
50%,27.066250,27.370001,26.850000,27.109688,19.065201,5.097900e+07
75%,43.784375,44.227187,43.181561,43.867499,33.587988,7.156432e+07
max,351.320007,351.470001,342.200012,348.100006,348.100006,1.031789e+09


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9394 entries, 1986-03-13 00:00:00-05:00 to 2023-06-21 00:00:00-04:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       9394 non-null   float64
 1   High       9394 non-null   float64
 2   Low        9394 non-null   float64
 3   Close      9394 non-null   float64
 4   Adj Close  9394 non-null   float64
 5   Volume     9394 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 513.7 KB


In [45]:
fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

fig.show()

In [47]:
closeData = data['Close']
closeData

Date
1986-03-13 00:00:00-05:00      0.097222
1986-03-14 00:00:00-05:00      0.100694
1986-03-17 00:00:00-05:00      0.102431
1986-03-18 00:00:00-05:00      0.099826
1986-03-19 00:00:00-05:00      0.098090
                                ...    
2023-06-14 00:00:00-04:00    337.339996
2023-06-15 00:00:00-04:00    348.100006
2023-06-16 00:00:00-04:00    342.329987
2023-06-20 00:00:00-04:00    338.049988
2023-06-21 00:00:00-04:00    334.290009
Name: Close, Length: 9394, dtype: float64

In [48]:
closeData.plot()

In [53]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_close_data = scaler.fit_transform(closeData.to_numpy().reshape(-1,1))
scaled_close_data

array([[1.99534673e-05],
       [2.99302009e-05],
       [3.49214366e-05],
       ...,
       [9.83419948e-01],
       [9.71121444e-01],
       [9.60317208e-01]])

In [214]:
training_size = int( len(scaled_close_data) * .85)
train_data, test_data = scaled_close_data[:training_size], scaled_close_data[training_size:]

In [215]:
fig = go.Figure()
fig.add_trace( 
              go.Scatter(x = list(range(0, training_size)), y = train_data.reshape(1, -1)[0], mode='lines', name = 'train data')
              ) 

fig.add_trace( 
              go.Scatter(x = list(range(training_size, len(data))), y = test_data.reshape(1, -1)[0], mode='lines', name = 'test data')
              ) 
fig.update_layout(title = 'train and test data')
fig.show()


In [216]:
def creatDataset(data, timrstep = 1):
    X, y = [], []
    
    limit = len(data) - timrstep 
    for i in range(limit):
        X.append(data[i: timrstep + i, 0])
        y.append(data[i + timrstep, 0])
        
    return np.array(X), np.array(y)

In [217]:
timestep = 100
X_train,y_train = creatDataset(train_data, timestep)
X_test,y_test = creatDataset(test_data, timestep)


X_train = X_train.reshape(*X_train.shape, 1)
X_test = X_test.reshape(*X_test.shape, 1)
print(X_train.shape, X_test.shape)

(7884, 100, 1) (1310, 100, 1)


In [218]:
print(len(data), X_train.shape[0] + X_test.shape[0]+200)

9394 9394


In [228]:
model = Sequential()

model.add( LSTM(75, return_sequences=True, input_shape = (100, 1)) )
model.add( LSTM(75, return_sequences=True) )
model.add( LSTM(75) )
model.add( Dense(1) )

model.compile( loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
247/247 [==============================] - 21s 71ms/step - loss: 1.2067e-04
Epoch 2/20
247/247 [==============================] - 17s 70ms/step - loss: 1.9230e-05
Epoch 3/20
247/247 [==============================] - 18s 72ms/step - loss: 1.5137e-05
Epoch 4/20
247/247 [==============================] - 18s 73ms/step - loss: 1.3168e-05
Epoch 5/20
247/247 [==============================] - 18s 74ms/step - loss: 1.3901e-05
Epoch 6/20
247/247 [==============================] - 18s 74ms/step - loss: 1.0690e-05
Epoch 7/20
247/247 [==============================] - 18s 74ms/step - loss: 1.0992e-05
Epoch 8/20
247/247 [==============================] - 18s 73ms/step - loss: 9.7208e-06
Epoch 9/20
247/247 [==============================] - 18s 73ms/step - loss: 9.2590e-06
Epoch 10/20
247/247 [==============================] - 18s 74ms/step - loss: 8.5465e-06
Epoch 11/20
247/247 [==============================] - 18s 74ms/step - loss: 7.3040e-06
Epoch 12/20
247/247 [====================

In [229]:
loss = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')

41/41 [==============================] - 2s 24ms/step - loss: 1.7151e-04
Loss: 0.00017150622443296015


In [230]:
train_pre = model.predict(X_train)
test_pre = model.predict(X_test)

41/41 [==============================] - 1s 24ms/step


In [231]:
train_pre_data = scaler.inverse_transform(train_pre)
test_pre_data = scaler.inverse_transform(test_pre)

In [232]:
train_limit = [timestep, len(train_pre_data) + timestep]
limi_for_pri_test = (train_limit[1] + timestep, 
                     train_limit[1] + timestep + len(test_pre_data))

fig = go.Figure()
fig.add_trace( 
              go.Scatter(
                  x = list(range(len(data['Close'].to_numpy()))),
                  y = data['Close'].to_numpy(),
                  mode='lines', 
                  name = 'All Real Data')
              ) 

fig.add_trace( 
              go.Scatter(
                  x = list(range(*train_limit)),
                  y = train_pre_data.reshape(1, -1)[0], 
                  mode='lines', 
                  name = 'train pre data')
              ) 

fig.add_trace( 
              go.Scatter(
                  x = list(range(*limi_for_pri_test)),
                  y = test_pre_data.reshape(1, -1)[0],
                  mode='lines', 
                  name = 'test pre data')
              ) 
fig.update_layout(title = f'train and test Prediction data<br>with loss: {loss:.5f}')
fig.show()